# 🧼 01_clean_afdr_data.ipynb

## 🎯 Purpose

This notebook prepares and cleans the dataset `1_datasets/raw_data/afdr_a8.csv`, 
which comes from the USDA Agricultural Finance Databook 
(Table A8). The dataset includes loan volume data across 
various loan sizes for U.S. farms, reported quarterly.

## 🤔 Why this data is relevant

Our project studies the relationship between demographic and 
financial behavior features and the likelihood of loan default, 
with a focus on comparing traditional loans and Buy Now, 
Pay Later (BNPL) services. 

This dataset offers historical insights into how traditional 
loan systems are structured and categorized by size, 
giving us a useful point of comparison with modern 
BNPL patterns.

## 📁 Output

The final cleaned data will be saved in:
- `../1_datasets/processed_datasets/afdr_cleaned.csv`


In [1]:
# Import library for data cleaning
import pandas as pd

In [7]:
#  Load the raw dataset from the 1_datasets/raw_data folder
df = pd.read_csv("../1_datasets/raw_data/afdr_a8.csv")

# View the top 5 rows to understand structure
df.head()

,Loan Characteristic,Period,All sizes,"$1,000 to!$9,000","$10,000 to!$24,000","$25,000 to!$49,000","$50,000 to!$99,000","$100,000 to!$249,000","$250,000!and over"
0,Volume of loans (thousands of dollars),1999Q4,1051926.0,63260.0,112099.0,126997.0,174266.0,236799.0,338506
1,Volume of loans (thousands of dollars),2000Q1,1333957.0,109035.0,169336.0,145658.0,191837.0,326655.0,391436
2,Volume of loans (thousands of dollars),2000Q2,922761.0,79545.0,113414.0,137081.0,123415.0,199123.0,270183
3,Volume of loans (thousands of dollars),2000Q3,926568.0,83900.0,133918.0,123899.0,121227.0,169822.0,293803
4,Volume of loans (thousands of dollars),2000Q4,922761.0,79545.0,113414.0,137081.0,123415.0,199123.0,270183


In [8]:
# Split 'Period' into 'Year' and 'Quarter'
df[["Year", "Quarter"]] = df["Period"].str.extract(r"(\d{4})Q(\d)")
df["Year"] = df["Year"].astype(int)
df["Quarter"] = df["Quarter"].astype(int)

# Move 'Year' and 'Quarter' to the front for clarity
cols = ["Year", "Quarter"] + [
    col for col in df.columns if col not in ["Year", "Quarter"]
]
df = df[cols]

# Drop the 'Loan Characteristic' and 'Period' columns
df = df.drop(columns=["Loan Characteristic", "Period"])

# View the top 5 rows to observe the changes
df.head()

,Year,Quarter,All sizes,"$1,000 to!$9,000","$10,000 to!$24,000","$25,000 to!$49,000","$50,000 to!$99,000","$100,000 to!$249,000","$250,000!and over"
0,1999,4,1051926.0,63260.0,112099.0,126997.0,174266.0,236799.0,338506
1,2000,1,1333957.0,109035.0,169336.0,145658.0,191837.0,326655.0,391436
2,2000,2,922761.0,79545.0,113414.0,137081.0,123415.0,199123.0,270183
3,2000,3,926568.0,83900.0,133918.0,123899.0,121227.0,169822.0,293803
4,2000,4,922761.0,79545.0,113414.0,137081.0,123415.0,199123.0,270183


## 🔎 Variable Dictionary

| Column              | Description                                      |
|---------------------|--------------------------------------------------|
| Year                | Year of the survey                              |
| Quarter             | Quarter of the year
| All sizes           | Total volume of all loans in that quarter       |
| $1,000 to!$9,000    | Loan volume between $1k and $9k                 |
| $10,000 to!$24,000  | Loan volume between $10k and $24k              |
| $25,000 to!$49,000  | Loan volume between $25k and $49k              |
| $50,000 to!$99,000  | Loan volume between $50k and $99k              |
| $100,000 to!$249,000| Loan volume between $100k and $249k            |
| $250,000!and over   | Loan volume over $250k                          |

In [9]:
# Rename messy columns to be cleaner and consistent
df.rename(
    columns={
        "$1,000 to!$9,000": "$1k–9k",
        "$10,000 to!$24,000": "$10k–24k",
        "$25,000 to!$49,000": "$25k–49k",
        "$50,000 to!$99,000": "$50k–99k",
        "$100,000 to!$249,000": "$100k–249k",
        "$250,000!and over": "$250k+",
        "All sizes": "All_Loan_Sizes",
    },
    inplace=True,
)

# Check new column names
df.columns

Index(['Year', 'Quarter', 'All_Loan_Sizes', '$1k–9k', '$10k–24k', '$25k–49k',
       '$50k–99k', '$100k–249k', '$250k+'],
      dtype='object')

In [12]:
# Identify which columns to convert (skip 'Period')
cols_to_convert = df.columns.difference(["Year", "Quarter"])

# Convert selected columns to numeric type
df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors="coerce")

# Confirm types
df.dtypes

Year                int64
Quarter             int64
All_Loan_Sizes    float64
$1k–9k            float64
$10k–24k          float64
$25k–49k          float64
$50k–99k          float64
$100k–249k        float64
$250k+            float64
dtype: object

In [11]:
# Check for missing values
missing = df[df.isnull().any(axis=1)]
print("Rows with missing values:", len(missing))

# Replace missing values with 0
df.fillna(0, inplace=True)

Rows with missing values: 0


## Save Cleaned Data

We now export the cleaned dataset to the 
`/1_datasets/processed_datasets` folder. This cleaned version will be 
used in Milestone 4 for further exploration and 
comparative modeling.

In [14]:
# 💾 Save cleaned CSV for use in MS4
df.to_csv("../1_datasets/processed_datasets/afdr_cleaned.csv", index=False)

print("✅ Cleaned file saved in /1_datasets/processed_datasets/")

✅ Cleaned file saved in /1_datasets/processed_datasets/
